In [1]:
// Helper functions
const log = <T extends unknown[]>(...data: T) => (console.log(...data), data);
const mod = (a: number, b: number) => ((a % b) + b) % b;

In [2]:
// Preprocess data
const exampleRawData = `
[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
`.slice(1, -1);

const rawData = await Deno.readTextFile("day_10.txt");

function preprocess(rawData: string) {
  return rawData
    .split("\n")
    .map((e) => [...e.match(/(?<=[\[\(\{]).+?(?=[\]\)\}])/g)!] as string[])
    .map(([lights, ...other]) => ({
      lights: lights.split("").map(e => e === "." ? false : true),
      joltages: other.pop()!.split(",").map(Number),
      buttonsSet: other.map(e => e.split(",").map(Number)),
    }))
}

In [14]:
function part1(data: ReturnType<typeof preprocess>) {
  let total = 0;
  for (const { lights, buttonsSet } of data) {
    console.log(lights, buttonsSet);
    const state: boolean[] = new Array(lights.length).fill(false);
    let states = [state];
    outer: for (let presses = 1; presses < 10; presses++) {
      let nextStates: typeof states = [];
      for (const rawState of states) {
        for (const buttons of buttonsSet) {
          const newState = [...rawState];
          let allBad = true;
          for (const i of buttons) {
            newState[i] = !newState[i];
            if (newState[i] === lights[i]) allBad = false;
          }
          if (newState.every((e,i) => e === lights[i])) {
            console.log(presses);
            total += presses;
            break outer;
          }
          if (!allBad) nextStates.push(newState);
        }
        states = nextStates;
      }
      // let bestScore = -1000;
      // let bestButtons = buttonsSet[0];
      // let bestStates: boolean[][] = [];
      // for (const buttons of buttonsSet) {
      //   const newState = [...state];
      //   let score = 0;
      //   for (const i of buttons) {
      //     newState[i] = !newState[i];
      //     score += newState[i] === lights[i] ? 1 : -1;
      //   }
      //   if (score > bestScore) {
      //     score = bestScore;
      //     bestButtons = buttons;
      //     bestStates = [newState];
      //     console.log(newState);
      //   }
      // }
      // state = bestState;
      // presses += 1;
      // if (state.every((e,i) => e === lights[i])) {
      //   console.log(presses);
      //   total += presses;
      //   break;
      // }
    }
  }
  return total;
}

part1(
  preprocess(
    // exampleRawData,
    rawData,
  )
)

[
  false, true, true,
  false, true, false,
  false, true, true,
  false
] [
  [ 3, 6 ],
  [
    0, 1, 2, 3,
    4, 5, 7, 9
  ],
  [
    0, 1, 5, 6,
    7, 8, 9
  ],
  [ 1, 9 ],
  [
    0, 1, 3, 4,
    5, 6, 7
  ],
  [ 0, 1, 2, 3, 4, 5 ],
  [
    1, 2, 3, 4,
    5, 6, 7, 8
  ],
  [ 2, 3, 5, 7, 8 ],
  [ 2, 3, 5, 7, 9 ],
  [
    0, 1, 2, 3,
    4, 6, 9
  ],
  [ 4, 5, 6, 7, 8 ],
  [ 3, 6, 7, 8, 9 ]
]
4
[ false, false, true, true, false ] [ [ 2, 4 ], [ 0, 4 ], [ 1, 3, 4 ], [ 0, 2, 3, 4 ], [ 2, 3 ] ]
1
[
  false, false,
  true,  false,
  true,  true,
  false, false
] [
  [ 1, 2, 3, 4, 5, 6 ],
  [ 0, 2, 7 ],
  [ 1, 2, 3 ],
  [ 0, 4 ],
  [ 0, 1, 4, 5, 6, 7 ],
  [ 0, 4, 5, 7 ],
  [ 0, 1, 2, 3, 5, 7 ]
]
2
[
  true,  false, true,
  false, true,  false,
  false, true,  true
] [
  [
    0, 2, 4, 5,
    6, 7, 8
  ],
  [ 0, 2, 4, 8 ],
  [ 0, 1, 4, 5 ],
  [ 3, 6, 8 ],
  [ 2, 6, 8 ],
  [ 0, 1, 2, 3, 5, 6 ],
  [ 0, 5, 7 ],
  [ 0, 3, 4, 5 ],
  [ 2, 4, 5, 6, 8 ],
  [ 0, 1, 3, 5 ]
]
5
[ true, true, true,

449

In [3]:
function part2(data: ReturnType<typeof preprocess>) {
  let total = 0;
  for (const { buttonsSet, joltages } of data) {
    console.log(buttonsSet, joltages);
    buttonsSet.sort((a,b)=>b.length-a.length);

    const cache: Map<string, number> = new Map();
    function dfs(joltageState: number[], startI = 0): number {
      const key = joltageState.join(",");
      if (cache.has(key)) return cache.get(key)!;
      let min = Infinity;
      buttons: for (let i = startI; i < buttonsSet.length; i++) {
        const buttons = buttonsSet[i];
        const newJoltages = [...joltageState];
        for (const i of buttons) {
          newJoltages[i] -= 1;
          if (newJoltages[i] < 0) continue buttons;
        }
        if (Math.max(...newJoltages) === 0) {
          // console.log(buttons);
          cache.set(key, 1);
          return 1;
        }
        const res = dfs(newJoltages, i);
        if (res) {
          // console.log(buttons);
          cache.set(key, res + 1);
          return res + 1;
          min = res + 1 < min ? res + 1 : min;
        }
      }
      cache.set(key, 0);
      return 0;
      // return min;
    }
    total += log(dfs(joltages))[0];
  }
  return total;
}

part2(
  preprocess(
    // exampleRawData,
    rawData,
  )
)


[
  [ 3, 6 ],
  [
    0, 1, 2, 3,
    4, 5, 7, 9
  ],
  [
    0, 1, 5, 6,
    7, 8, 9
  ],
  [ 1, 9 ],
  [
    0, 1, 3, 4,
    5, 6, 7
  ],
  [ 0, 1, 2, 3, 4, 5 ],
  [
    1, 2, 3, 4,
    5, 6, 7, 8
  ],
  [ 2, 3, 5, 7, 8 ],
  [ 2, 3, 5, 7, 9 ],
  [
    0, 1, 2, 3,
    4, 6, 9
  ],
  [ 4, 5, 6, 7, 8 ],
  [ 3, 6, 7, 8, 9 ]
] [
  52, 67, 66, 109, 49,
  65, 70, 66,  33, 72
]


RangeError: Map maximum size exceeded

In [ ]:
// Attempt 2
function aGreaterThanB(a: number[], b: number[]) {
  for (let i = 0; i < a.length; i++) {
    if (a[i] > b[i]) return true;
  }
  return false;
}
function part2(data: ReturnType<typeof preprocess>) {
  let total = 0;
  for (const { buttonsSet, joltages } of data) {
    console.log(buttonsSet, joltages);
    buttonsSet.sort((a,b)=>b.length-a.length);
    const indexes = Array.from(joltages, (i) => Array.from({ length: i }, (j) => ));
  }
  return total;
}

part2(
  preprocess(
    // exampleRawData,
    rawData,
  )
)
